In [ ]:
print('Setting UP')
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
from sklearn.model_selection import train_test_split
from utlis import *

#### STEP 1: Initialize Data

In [ ]:
path = 'DataCollected'
data = importDataInfo(path)
print(data.head())

#### STEP 2: Visualize and Balance Data

In [ ]:
data = balanceData(data,display=True)

#### STEP 3 - Prepare for processing

In [4]:
imagesPath, steerings = loadData(path,data)

#### STEP 4 - Split train and validation data

In [ ]:
xTrain, xVal, yTrain, yVal = train_test_split(imagesPath, steerings,
                                              test_size=0.2,random_state=10)
print('Total Training Images: ',len(xTrain))
print('Total Validation Images: ',len(xVal))

#### STEP 5: Training

In [ ]:
model = createModel()
history = model.fit(dataGen(xTrain, yTrain, 100, 1),
                                  steps_per_epoch=100,
                                  epochs=20,
                                  validation_data=dataGen(xVal, yVal, 50, 0),
                                  validation_steps=50)

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['Training', 'Validation'])
plt.title('Loss')
plt.xlabel('Epoch')
plt.show()

In [12]:
model.save('model5.h5')

#### STEP 6: Convert Keras Model to Tensorflow-Lite Model

In [ ]:
import tensorflow as tf
new_model = tf.keras.models.load_model('model3.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(new_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model_quantize = converter.convert()
with open("model_3.tflite", "wb") as f:
    f.write(tflite_model_quantize)